# Dynamic Prompts & Long-Term Memory for HR Agents - LangChain 1.0

**Module:** Context Engineering - Dynamic Prompts & Persistent Memory

**Based on:** https://docs.langchain.com/oss/python/langchain/context-engineering

**What you'll learn:**
- 🎯 **Dynamic Prompts** with `@dynamic_prompt` middleware
- 📚 **Read** long-term memory in tools
- ✍️ **Write** long-term memory in tools
- 💾 **Store** user preferences across sessions
- 🔍 **Query** stored memories with namespaces
- 🎨 **Combine** both for personalized experiences

**HR Use Cases:**
- Personalized greetings based on stored preferences
- Remember employee communication styles
- Store and retrieve work schedule preferences
- Track career goals and development plans

**Time:** 2-3 hours

---

## Setup: Install Dependencies

In [ ]:
!pip install --pre -U langchain langchain-openai langgraph
!pip install langgraph-checkpoint-sqlite

## Setup: Configure API Key

In [ ]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print("✅ API Key configured!")

## Import Libraries

In [ ]:
from typing import Annotated
from dataclasses import dataclass

# Core LangChain
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig

# LangGraph
from langgraph.store.memory import InMemoryStore

# Utilities
from datetime import datetime
import json
import logging

# Enable debug logging
logging.basicConfig(level=logging.INFO)

print("✅ Imports successful!")

## Setup: Employee Database

In [ ]:
# Basic employee database
EMPLOYEE_DB = {
    "EMP101": {
        "name": "Priya Sharma",
        "department": "Engineering",
        "role": "Senior Developer",
        "leave_balance": 12
    },
    "EMP102": {
        "name": "Rahul Verma",
        "department": "Engineering",
        "role": "Engineering Manager",
        "leave_balance": 8
    },
    "EMP103": {
        "name": "Anjali Patel",
        "department": "HR",
        "role": "HR Director",
        "leave_balance": 15
    }
}

print(f"✅ Loaded {len(EMPLOYEE_DB)} employees")

---
# Part 1: Long-Term Memory Store Setup

**First, let's set up the memory store and verify it works correctly.**

## Step 1: Initialize Memory Store with Sample Data

In [ ]:
# Create in-memory store (in production, use PostgresStore or other persistent storage)
memory_store = InMemoryStore()

# Pre-populate with sample preferences
# Namespace format: ("category", "identifier")

# Employee 1: Priya - Communication preferences
memory_store.put(
    namespace=("employee_preferences", "EMP101"),
    key="communication_style",
    value={
        "style": "detailed",
        "language": "English",
        "response_format": "paragraphs",
        "last_updated": "2025-01-15"
    }
)

# Employee 1: Priya - Notification settings
memory_store.put(
    namespace=("employee_preferences", "EMP101"),
    key="notification_settings",
    value={
        "email_notifications": True,
        "slack_notifications": True,
        "sms_notifications": False,
        "frequency": "daily"
    }
)

# Employee 1: Priya - Work schedule
memory_store.put(
    namespace=("employee_preferences", "EMP101"),
    key="work_schedule",
    value={
        "preferred_hours": "9AM-5PM",
        "flexible": True,
        "remote_days": ["Monday", "Wednesday", "Friday"],
        "timezone": "Asia/Kolkata"
    }
)

# Employee 2: Rahul - Different preferences
memory_store.put(
    namespace=("employee_preferences", "EMP102"),
    key="communication_style",
    value={
        "style": "concise",
        "language": "English",
        "response_format": "bullet_points"
    }
)

memory_store.put(
    namespace=("employee_preferences", "EMP102"),
    key="notification_settings",
    value={
        "email_notifications": True,
        "slack_notifications": False,
        "sms_notifications": True,
        "frequency": "real-time"
    }
)

# Career goals
memory_store.put(
    namespace=("employee_career", "EMP101"),
    key="goals",
    value={
        "short_term": ["Learn Kubernetes", "Complete AWS certification"],
        "long_term": ["Become Tech Lead", "Mentor junior developers"],
        "last_updated": "2025-01-10"
    }
)

print("✅ Memory store initialized!")
print("\nStored preferences:")
print("  • EMP101: communication_style, notification_settings, work_schedule, career goals")
print("  • EMP102: communication_style, notification_settings")

## Step 2: Verify Store Data

**Let's verify the data is actually stored correctly before creating tools**

In [ ]:
print("=" * 70)
print("VERIFYING STORE DATA")
print("=" * 70)

# Check EMP101 preferences
namespace = ("employee_preferences", "EMP101")
print(f"\n📂 Namespace: {namespace}")

items = list(memory_store.search(namespace))
print(f"✅ Found {len(items)} items:")

for item in items:
    print(f"\n  🔑 Key: {item.key}")
    print(f"  📄 Value: {json.dumps(item.value, indent=4)}")

# Try direct get
print("\n" + "=" * 70)
print("DIRECT GET TEST")
print("=" * 70)

item = memory_store.get(namespace, "notification_settings")
if item:
    print(f"✅ Found notification_settings:")
    print(json.dumps(item.value, indent=2))
else:
    print("❌ notification_settings not found")

print("\n✅ Store verification complete!")

---
# Part 2: Context Schema & Dynamic Prompt

**Now let's set up the dynamic prompt that uses memory**

## Step 1: Define Context Schema

In [ ]:
# Define context schema that will be passed at runtime
@dataclass
class HRContext:
    """Runtime context for HR agent."""
    employee_id: str

print("✅ Context schema defined: HRContext(employee_id)")

## Step 2: Create @dynamic_prompt Middleware

This function generates personalized prompts using:
1. **Runtime context** - Employee ID
2. **Long-term memory** - Stored preferences
3. **Session state** - Conversation length

In [ ]:
@dynamic_prompt
def personalized_hr_prompt(request: ModelRequest) -> str:
    """
    Generate personalized HR prompt based on:
    - Runtime context (employee_id)
    - Long-term memory (preferences)
    - Session state (message count)
    """
    # 1. Access runtime context
    employee_id = request.runtime.context.employee_id
    
    # 2. Access long-term memory store
    store = request.runtime.store
    
    # 3. Access session state
    message_count = len(request.state.get("messages", []))
    
    # Look up employee info from database
    employee = EMPLOYEE_DB.get(employee_id, {})
    
    if not employee:
        return "You are a helpful HR assistant."
    
    name = employee.get("name", "Employee")
    department = employee.get("department", "Unknown")
    role = employee.get("role", "Unknown")
    leave_balance = employee.get("leave_balance", 0)
    
    # Time-based greeting
    hour = datetime.now().hour
    if hour < 12:
        greeting = "Good morning"
    elif hour < 17:
        greeting = "Good afternoon"
    else:
        greeting = "Good evening"
    
    # Base prompt
    prompt = f"""{greeting}! You are an AI HR assistant helping {name} ({employee_id}).

**Employee Context:**
- Name: {name}
- Department: {department}
- Role: {role}
- Leave Balance: {leave_balance} days
"""
    
    # Try to get communication preferences from long-term memory
    try:
        namespace = ("employee_preferences", employee_id)
        comm_prefs = store.get(namespace, "communication_style")
        
        if comm_prefs:
            style = comm_prefs.value.get("style", "balanced")
            response_format = comm_prefs.value.get("response_format", "paragraphs")
            
            prompt += f"\n**Communication Preferences (from memory):**"
            prompt += f"\n- Style: {style}"
            prompt += f"\n- Format: {response_format}"
            
            # Add style-specific instructions
            if style == "concise":
                prompt += "\n\n**Instructions:** Provide brief, to-the-point responses. Use bullet points."
            elif style == "detailed":
                prompt += "\n\n**Instructions:** Provide comprehensive explanations with examples."
            elif style == "formal":
                prompt += "\n\n**Instructions:** Maintain professional and formal tone."
        else:
            prompt += "\n\n**Instructions:** Be helpful and professional."
    
    except Exception as e:
        prompt += "\n\n**Instructions:** Be helpful and professional."
    
    # Add conversation length context
    if message_count > 10:
        prompt += "\n\n*Note: This is a long conversation. Focus on recent context and be concise.*"
    
    return prompt

print("✅ Dynamic prompt middleware created!")
print("\nThis prompt will:")
print("  • Greet user based on time of day")
print("  • Include employee context")
print("  • Load communication preferences from memory")
print("  • Adapt based on conversation length")

---
# Part 3: Create Tools with Closure Pattern

**Using closure pattern to avoid InjectedStore issues**

This pattern captures the store at tool creation time, making tools more reliable.

In [ ]:
def create_memory_tools(store: InMemoryStore):
    """
    Factory function that creates tools with store captured in closure.
    This avoids InjectedStore issues and is more reliable.
    """
    
    # READ TOOLS
    
    @tool
    def get_notification_settings(config: RunnableConfig) -> str:
        """Get notification settings for the current employee from long-term memory."""
        employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
        employee = EMPLOYEE_DB.get(employee_id, {})
        name = employee.get("name", "Employee")
        
        namespace = ("employee_preferences", employee_id)
        item = store.get(namespace, "notification_settings")
        
        if item:
            return f"""✅ **{name}'s Notification Settings:**

```json
{json.dumps(item.value, indent=2)}
```

*Retrieved from long-term memory*"""
        return f"❌ No notification settings found for {name}"
    
    @tool
    def get_communication_style(config: RunnableConfig) -> str:
        """Get communication style preference from long-term memory."""
        employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
        employee = EMPLOYEE_DB.get(employee_id, {})
        name = employee.get("name", "Employee")
        
        namespace = ("employee_preferences", employee_id)
        item = store.get(namespace, "communication_style")
        
        if item:
            return f"""✅ **{name}'s Communication Style:**

```json
{json.dumps(item.value, indent=2)}
```

*Retrieved from long-term memory*"""
        return f"❌ No communication style found for {name}"
    
    @tool
    def get_work_schedule(config: RunnableConfig) -> str:
        """Get work schedule preferences from long-term memory."""
        employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
        employee = EMPLOYEE_DB.get(employee_id, {})
        name = employee.get("name", "Employee")
        
        namespace = ("employee_preferences", employee_id)
        item = store.get(namespace, "work_schedule")
        
        if item:
            return f"""✅ **{name}'s Work Schedule:**

```json
{json.dumps(item.value, indent=2)}
```

*Retrieved from long-term memory*"""
        return f"❌ No work schedule found for {name}"
    
    @tool
    def get_all_preferences(config: RunnableConfig) -> str:
        """Get ALL preferences for the current employee from long-term memory."""
        employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
        employee = EMPLOYEE_DB.get(employee_id, {})
        name = employee.get("name", "Employee")
        
        namespace = ("employee_preferences", employee_id)
        items = list(store.search(namespace))
        
        if items:
            result = f"**All preferences for {name}:**\n\n"
            
            for item in items:
                result += f"### {item.key.replace('_', ' ').title()}\n"
                result += f"```json\n{json.dumps(item.value, indent=2)}\n```\n\n"
            
            result += f"*Total: {len(items)} preference sets*"
            return result
        return f"No preferences found for {name}."
    
    @tool
    def get_career_goals(config: RunnableConfig) -> str:
        """Get career development goals from long-term memory."""
        employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
        employee = EMPLOYEE_DB.get(employee_id, {})
        name = employee.get("name", "Employee")
        
        namespace = ("employee_career", employee_id)
        item = store.get(namespace, "goals")
        
        if item:
            goals = item.value
            result = f"**{name}'s Career Goals:**\n\n"
            result += f"**Short-term goals:**\n"
            for goal in goals.get("short_term", []):
                result += f"  • {goal}\n"
            result += f"\n**Long-term goals:**\n"
            for goal in goals.get("long_term", []):
                result += f"  • {goal}\n"
            result += f"\n*Last updated: {goals.get('last_updated', 'Unknown')}*"
            return result
        return f"No career goals set for {name}."
    
    # WRITE TOOLS
    
    @tool
    def update_notification_settings(
        email: Annotated[bool, "Enable email notifications"],
        slack: Annotated[bool, "Enable Slack notifications"],
        frequency: Annotated[str, "Frequency: daily, weekly, real-time"],
        config: RunnableConfig
    ) -> str:
        """Update notification settings in long-term memory."""
        employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
        employee = EMPLOYEE_DB.get(employee_id, {})
        name = employee.get("name", "Employee")
        
        new_settings = {
            "email_notifications": email,
            "slack_notifications": slack,
            "frequency": frequency,
            "last_updated": datetime.now().isoformat()
        }
        
        namespace = ("employee_preferences", employee_id)
        store.put(namespace, "notification_settings", new_settings)
        
        return f"""✅ **Updated notification settings for {name}:**

```json
{json.dumps(new_settings, indent=2)}
```

*Saved to long-term memory*"""
    
    @tool
    def add_career_goal(
        goal: Annotated[str, "Career goal to add"],
        term: Annotated[str, "short_term or long_term"],
        config: RunnableConfig
    ) -> str:
        """Add a career goal to long-term memory."""
        employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
        employee = EMPLOYEE_DB.get(employee_id, {})
        name = employee.get("name", "Employee")
        
        if term not in ["short_term", "long_term"]:
            return "❌ Term must be 'short_term' or 'long_term'"
        
        namespace = ("employee_career", employee_id)
        existing = store.get(namespace, "goals")
        
        if existing:
            goals = existing.value.copy()
        else:
            goals = {"short_term": [], "long_term": []}
        
        if goal not in goals[term]:
            goals[term].append(goal)
            goals["last_updated"] = datetime.now().isoformat()
            store.put(namespace, "goals", goals)
            
            return f"✅ Added {term.replace('_', '-')} goal for {name}: '{goal}'\n\n*Saved to long-term memory*"
        return f"⚠️ Goal '{goal}' already exists"
    
    return [
        # Read tools
        get_notification_settings,
        get_communication_style,
        get_work_schedule,
        get_all_preferences,
        get_career_goals,
        # Write tools
        update_notification_settings,
        add_career_goal
    ]

# Create tools with store captured in closure
memory_tools = create_memory_tools(memory_store)

print("✅ Memory tools created using closure pattern!")
print(f"\nCreated {len(memory_tools)} tools:")
for t in memory_tools:
    print(f"  • {t.name}")

## Create Basic Tools

In [ ]:
@tool
def check_leave_balance(config: RunnableConfig) -> str:
    """Check the current employee's leave balance."""
    employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
    employee = EMPLOYEE_DB.get(employee_id, {})
    
    if employee:
        name = employee.get("name")
        balance = employee.get("leave_balance")
        return f"{name} has {balance} days of leave remaining."
    return "Employee not found."

print("✅ Basic tools created!")

---
# Part 4: Create the Complete Agent

In [ ]:
# Combine all tools
all_tools = memory_tools + [check_leave_balance]

# Create agent with dynamic prompt and memory tools
hr_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=all_tools,
    middleware=[personalized_hr_prompt],  # ✅ Dynamic prompt
    context_schema=HRContext,              # ✅ Context schema
    store=memory_store                     # ✅ Store for dynamic prompt
)

print("✅ Complete HR Agent created!")
print("\nFeatures:")
print("  🎯 Dynamic prompts - Personalized based on user and memory")
print("  📚 Long-term memory - Read preferences")
print("  ✍️ Memory updates - Write preferences")
print("  🔍 Memory search - Find all preferences")
print("  🗂️ Closure pattern - Reliable tool execution")

---
# Part 5: Testing the Complete System

## Test 1: Dynamic Prompt with Memory-Based Personalization

In [ ]:
print("=" * 70)
print("TEST 1: Priya (Detailed Style from Memory)")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! How many leave days do I have?"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")
print("\n💡 Notice: Response is detailed because Priya's preference (from memory) is 'detailed'")

## Test 2: Different User with Different Preferences

In [ ]:
print("\n" + "=" * 70)
print("TEST 2: Rahul (Concise Style from Memory)")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! How many leave days do I have?"}]},
    context=HRContext(employee_id="EMP102")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")
print("\n💡 Notice: Response is concise because Rahul's preference (from memory) is 'concise'")

## Test 3: Read Preferences from Memory

In [ ]:
print("\n" + "=" * 70)
print("TEST 3: Retrieve Notification Settings from Long-Term Memory")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "What are my notification settings?"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")
print("\n✅ Successfully retrieved from long-term memory!")

## Test 4: Get All Preferences

In [ ]:
print("\n" + "=" * 70)
print("TEST 4: Get All Stored Preferences")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Show me all my preferences"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")

## Test 5: Update Preferences (Write to Memory)

In [ ]:
print("\n" + "=" * 70)
print("TEST 5: Update Notification Settings in Long-Term Memory")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Turn off my email notifications and set frequency to weekly"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")

## Test 6: Verify Update Persisted

In [ ]:
print("\n" + "=" * 70)
print("TEST 6: Verify Changes Were Saved to Memory")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "What are my notification settings now?"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")
print("\n✅ Changes persisted in long-term memory!")

## Test 7: Career Goals

In [ ]:
print("\n" + "=" * 70)
print("TEST 7: Career Goals from Memory")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "What are my career goals?"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")

## Test 8: Add New Career Goal

In [ ]:
print("\n" + "=" * 70)
print("TEST 8: Add New Career Goal to Memory")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Add a short-term goal: Complete Python advanced certification"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")

# Verify it was added
print("\n" + "-" * 70)
print("Verification:")
result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "What are my career goals now?"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")
print("\n✅ New goal persisted in long-term memory!")

---
# Summary

## Key Patterns Learned

### 1. Dynamic Prompts with @dynamic_prompt
```python
@dynamic_prompt
def my_prompt(request: ModelRequest) -> str:
    # Access runtime context
    user_id = request.runtime.context.user_id
    
    # Access long-term memory
    store = request.runtime.store
    prefs = store.get(("prefs", user_id), "style")
    
    # Access session state
    msg_count = len(request.state["messages"])
    
    return f"Personalized prompt"

agent = create_agent(
    middleware=[my_prompt],
    context_schema=MyContext,
    store=memory_store
)
```

### 2. Closure Pattern for Tools (Recommended)
```python
def create_memory_tools(store: InMemoryStore):
    @tool
    def read_memory(config: RunnableConfig) -> str:
        user_id = config.get("configurable", {}).get("user_id")
        item = store.get(("category", user_id), "key")
        return item.value if item else "Not found"
    
    return [read_memory]

tools = create_memory_tools(memory_store)
```

### 3. Write to Memory
```python
@tool
def write_memory(config: RunnableConfig) -> str:
    user_id = config.get("configurable", {}).get("user_id")
    store.put(("category", user_id), "key", {"data": "value"})
    return "Saved!"
```

## Why Closure Pattern?

✅ **Advantages:**
- More reliable than InjectedStore
- Store is captured at tool creation time
- No dependency injection issues
- Easier to debug

⚠️ **InjectedStore Alternative:**
- Can still use `InjectedStore` if needed
- May have compatibility issues in some setups
- Closure pattern is production-proven

## Production Checklist

- [x] Dynamic prompts personalize experience
- [x] Long-term memory stores preferences
- [x] Tools read from memory (closure pattern)
- [x] Tools write to memory (closure pattern)
- [x] Namespaces organize data
- [ ] Use persistent storage (PostgreSQL, Redis)
- [ ] Add error handling and retries
- [ ] Implement data validation
- [ ] Add audit logging
- [ ] Set up backup strategy

---

**Congratulations!** You now have a working HR agent with dynamic prompts and long-term memory using the reliable closure pattern! 🎉